### Projeto 2-1 Visão Computacional
Elisa Malzoni

In [2]:
%reset -f
%matplotlib inline

import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt

import time

## Aula 09 - Good features to Track

In [ ]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
           qualityLevel = 0.3,
           minDistance = 7,
           blockSize = 7 )

color = np.random.randint(0,255,(100,3))

# Parametriza o Lucas-Kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


ret, previous = captura.read()

previous_gray = cv2.cvtColor(previous, cv2.COLOR_RGB2GRAY)
p0 = cv2.goodFeaturesToTrack(previous_gray, mask = None, **dt_params)
mask = np.zeros_like(previous)

fomv = 0

while(1):
    
    ret, actual = captura.read()
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_RGB2GRAY)
    
    # Calcula Fluxo Otico
    p1, st, err = cv2.calcOpticalFlowPyrLK(previous_gray, actual_gray, p0, None, **lk_params)
        
    # Seleciona somente os melhores pontos
    good_new = p1[st==1]
    good_old = p0[st==1]
       

    # Desenha as trilhas para cada ponto em p1 e p0
    for i,(new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), [0,0,255], 2)
        actual = cv2.circle(actual,(a,b),5,color[i].tolist(),-1)
    

    # Calcula fluxo otico medio
    sub = np.subtract(good_new, good_old)
    fom = np.mean(sub, axis=0) 
    fom = (-1)*fom
    fomv += fom
#     print(fomv)
    
    frame = cv2.add(actual, mask)
    framebw = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    rows, cols = framebw.shape
    
    tx = (np.uint8(fomv[0]))
    ty = (np.uint8(fomv[1]))
    
    M = np.float32([[1,0,tx],
                    [0,1,ty]])
    
    framet = cv2.warpAffine(frame, M,(cols,rows))
    
    cv2.imshow("Video", framet)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)

captura.release()
cv2.destroyAllWindows()

In [ ]:
captura.release()

## Dense Optical flow

In [1]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

color = np.random.randint(0,255,(100,3))

ret, previous = captura.read()

previous_gray = cv2.cvtColor(previous, cv2.COLOR_RGB2GRAY)
hsv = np.zeros_like(previous)

hsv[...,1] = 255

fomv = 0

while(1):
    
    ret, actual = captura.read()
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_RGB2GRAY)
    
    # Calcula Fluxo Otico
###
##
###
##
    previus_gray_center = #slicing para pegar o centro da imagem
    actual_gray_center = 
    flow = cv2.calcOpticalFlowFarneback(previous_gray_center, actual_gray_center,None, 0.5, 3, 15, 3, 5, 1.2, 0)       
    
    
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    frame = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
        
    cv2.imshow("Video", frame)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray.copy()
    
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)

captura.release()
cv2.destroyAllWindows()

SyntaxError: invalid syntax (<ipython-input-1-6b8b5fb90221>, line 24)

In [ ]:
captura.release()